### ** Agent-as-a-Tool  🧑‍🍳 **


MyLab_ADK_2 — MCP, Interoperability, and Orchestration

Part 0 — Setup & Authentication 🔑
Verify the Kernel

Make sure you’re using your correct venv

In [1]:
import sys, platform
print("Python:", platform.python_version())
print("Kernel: ", sys.executable)

Python: 3.12.3
Kernel:  /home/oem/Repos/CursoLLM/venv/bin/python


In [5]:
# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

api_key = getpass('Enter your Google API Key: ')

genai.configure(api_key=api_key)

os.environ['GOOGLE_API_KEY'] = api_key

genai.configure(api_key=api_key)
print("✅ API Key configured successfully! Let the fun begin.")

✅ All libraries are ready to go!
✅ API Key configured successfully! Let the fun begin.


## 2.2 The Agent-as-a-Tool: Consulting a Specialist 🧑‍🍳

Why build one agent that does everything when you can build a **team of specialist agents?** The **Agent-as-a-Tool** pattern 
allows one agent to delegate a task to another agent.

**Key Concept:** This is different from a sub-agent. When Agent A calls Agent B as a tool, 
Agent B's response is passed **back to Agent A**. Agent A then uses that information to form its own final 
response to the user. It's a powerful way to compose complex behaviors from simpler, focused, and reusable agents.

### How It Works

Our top-level agent, the `trip_data_concierge_agent`, acts as the **Orchestrator**. It has two tools at its disposal:

1.  `call_db_agent`: A function that internally calls our `db_agent` to fetch raw data.
2.  `call_concierge_agent`: A function that calls the `concierge_agent`.

The `concierge_agent` itself has a tool: the `food_critic_agent`.

The flow for a complex query is:

1.  **User** asks the `trip_data_concierge_agent` for a hotel and a nearby restaurant.
2.  The **Orchestrator** first calls `call_db_agent` to get hotel data.
3.  The data is saved in `tool_context.state`.
4.  The **Orchestrator** then calls `call_concierge_agent`, which retrieves the hotel data from the context.
5.  The `concierge_agent` receives the request and decides it needs to use its own tool, the `food_critic_agent`.
6.  The `food_critic_agent` provides a witty recommendation.
7.  The `concierge_agent` gets the critic's response and politely formats it.
8.  This final, polished response is returned to the **Orchestrator**, which presents it to the user.

In [6]:
import asyncio
from google.adk.tools import ToolContext
from google.adk.tools.agent_tool import AgentTool

# Assume 'db_agent' is a pre-defined NL2SQL Agent
# For this example, we'll create placeholder agents.

db_agent = Agent(
    name="db_agent",
    model="gemini-2.5-flash",
    instruction="You are a database agent. When asked for data, return this mock JSON object: {'status': 'success', 'data': [{'name': 'The Grand Hotel', 'rating': 5, 'reviews': 450}, {'name': 'Seaside Inn', 'rating': 4, 'reviews': 620}]}"
    )

# --- 1. Define the Specialist Agents ---

# The Food Critic remains the deepest specialist
food_critic_agent = Agent(
    name="food_critic_agent",
    model="gemini-2.5-flash",
    instruction="You are a snobby but brilliant food critic. You ONLY respond with a single, witty restaurant suggestion near the provided location.",
)

# The Concierge knows how to use the Food Critic
concierge_agent = Agent(
    name="concierge_agent",
    model="gemini-2.5-flash",
    instruction="You are a five-star hotel concierge. If the user asks for a restaurant recommendation, you MUST use the `food_critic_agent` tool. Present the opinion to the user politely.",
    tools=[AgentTool(agent=food_critic_agent)]
)


# --- 2. Define the Tools for the Orchestrator ---

async def call_db_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    Use this tool FIRST to connect to the database and retrieve a list of places, like hotels or landmarks.
    """
    print("--- TOOL CALL: call_db_agent ---")
    agent_tool = AgentTool(agent=db_agent)
    db_agent_output = await agent_tool.run_async(
        args={"request": question}, tool_context=tool_context
    )
    # Store the retrieved data in the context's state
    tool_context.state["retrieved_data"] = db_agent_output
    return db_agent_output


async def call_concierge_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    After getting data with call_db_agent, use this tool to get travel advice, opinions, or recommendations.
    """
    print("--- TOOL CALL: call_concierge_agent ---")
    # Retrieve the data fetched by the previous tool
    input_data = tool_context.state.get("retrieved_data", "No data found.")

    # Formulate a new prompt for the concierge, giving it the data context
    question_with_data = f"""
    Context: The database returned the following data: {input_data}

    User's Request: {question}
    """

    agent_tool = AgentTool(agent=concierge_agent)
    concierge_output = await agent_tool.run_async(
        args={"request": question_with_data}, tool_context=tool_context
    )
    return concierge_output


# --- 3. Define the Top-Level Orchestrator Agent ---

trip_data_concierge_agent = Agent(
    name="trip_data_concierge",
    model="gemini-2.5-flash",
    description="Top-level agent that queries a database for travel data, then calls a concierge agent for recommendations.",
    tools=[call_db_agent, call_concierge_agent],
    instruction="""
    You are a master travel planner who uses data to make recommendations.

    1.  **ALWAYS start with the `call_db_agent` tool** to fetch a list of places (like hotels) that match the user's criteria.

    2.  After you have the data, **use the `call_concierge_agent` tool** to answer any follow-up questions for recommendations, opinions, or advice related to the data you just found.
    """,
)

print(f"✅ Orchestrator Agent '{trip_data_concierge_agent.name}' is defined and ready.")

    


✅ Orchestrator Agent 'trip_data_concierge' is defined and ready.


In [7]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai.types import Content, Part
from IPython.display import Markdown, display

session_service = InMemorySessionService()
my_user_id = "user-001"

async def run_agent_query(agent, query: str, session, user_id: str):
    runner = Runner(agent=agent, session_service=session_service, app_name=agent.name)
    final = ""
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session.id,
        new_message=Content(parts=[Part(text=query)], role="user")
    ):
        if event.is_final_response():
            final = event.content.parts[0].text
    display(Markdown(final))
    return final




In [10]:
# --- Let's test the Trip Data Concierge Agent ---

async def run_trip_data_concierge():
    session = await session_service.create_session(
        app_name=trip_data_concierge_agent.name, 
        user_id=my_user_id
        )
    query = ("Find the top-rated hotels in Venice from the database, "
             "then suggest a dinner spot near the one with the most reviews.")
    print("User Query:", query)

    # We call our existing helper function with the top-level orchestrator agent
    await run_agent_query(trip_data_concierge_agent, query, session, my_user_id)

# Run the test:
await run_trip_data_concierge()

User Query: Find the top-rated hotels in Venice from the database, then suggest a dinner spot near the one with the most reviews.


The top-rated hotels in Venice are The Grand Hotel (5 stars, 450 reviews) and Seaside Inn (4 stars, 620 reviews). I recommend C&O Trattoria for a dinner spot near Seaside Inn. Their garlic knots are legendary!

## 🧩 Key Concepts Recap: Multi-Agent Orchestration with Shared Context

### **Agent**
An **Agent** is an autonomous reasoning unit that can:
- Understand instructions (via `instruction` or `description`).
- Call tools (functions, APIs, or other agents) to complete tasks.
- Use an underlying LLM (`model` parameter) to generate responses.

---

### **AgentTool**
- A **wrapper** that allows an Agent to be used **as a tool** by another Agent.
- Enables composition: one Agent can call another just like a function.
- Example: `concierge_agent` uses `food_critic_agent` via `AgentTool`.

---

### **Tool Function**
- A regular Python async function annotated with a docstring that describes:
  - What it does.
  - When it should be called.
- Registered in the top-level Agent's `tools` list.
- Can:
  - Call APIs.
  - Run other Agents.
  - Store intermediate results in `ToolContext`.

---

### **ToolContext**
- A temporary **shared state** object passed between tools **in a single user turn**.
- Behaves like a dictionary (`tool_context.state`).
- Used for:
  - Passing data from one tool to another.
  - Avoiding repeated API calls.
- **Ephemeral**: state is cleared after the user turn ends.

---

### **Orchestrator Agent**
- A **top-level agent** that coordinates multiple steps and tools.
- Holds the workflow logic:
  1. Call `call_db_agent` to fetch data.
  2. Call `call_concierge_agent` to get recommendations based on that data.
- Tools can be other agents (via `AgentTool`) or custom functions.

---

### **Session & SessionService**
- **Session**: Tracks conversation history and context over multiple turns.
- **SessionService**: Creates and manages sessions.
- In our example:
  - `session_service.create_session()` starts a fresh session.
  - We pass it to `run_agent_query()` so the agent can keep track of the conversation.

---

### **Execution Flow Recap**
1. **User query** → sent to `trip_data_concierge_agent`.
2. **First tool** (`call_db_agent`) runs → gets data → saves it in `tool_context.state`.
3. **Second tool** (`call_concierge_agent`) reads saved data → reformulates prompt → calls `concierge_agent`.
4. **Concierge agent** calls **food critic agent** internally via `AgentTool`.
5. **Final response** is returned to the user in a single step.

---

✅ **Key Benefits**:
- Modular: each Agent focuses on one skill.
- Reusable: Agents can be wrapped as tools.
- Context-passing: `ToolContext` avoids messy global variables.
- Clear orchestration: top-level Agent decides the sequence.

### ** Agent with a Memory - The Adaptive Planner 🗺️ **

In this part, we will build a **multi-day trip planner** that not only remembers previous conversation turns but also adapts to user feedback.  
The main difference from earlier parts is **memory management**: by reusing the same `Session` object across multiple turns, the agent maintains context and can adapt its plan accordingly.

---

### Concept: Why Memory Matters in Conversational Agents

When building conversational AI:
- **Without memory** → Each turn is independent, no context is carried forward.  
- **With memory** → The agent can recall past user inputs, previous outputs, and adapt based on feedback.  
- The **Session** object acts like the agent’s “short-term memory”, storing relevant state between turns.

---

### Agent Definition – Adaptive Multi-Day Trip Planner

In [ ]:
# --- Agent Definition: The Adaptive Planner ---

def create_multi_day_trip_agent():
    """ Create the PRogressive Multi-Day Planner agent"""
    return Agent(
        name="multi_day_trip_agent",
        model="gemini-2.5-flash",

        description="Agent that progressively plans a multi-day trip, remembering previous days and adapting to user feedback",
        instruction=
    )
